<h1>LLM로드</h1>

<h2>1. Local 동작 llama2</h2>

In [35]:
from langchain_community.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

llm = LlamaCpp(model_path="/Users/jominsu/Downloads/llama-2-7b-chat.Q8_0.gguf",
             temperature=0,
            # top_p=0.95,
            # max_tokens=8192,
            # verbose=True,
            # # n_ctx: 모델이 한 번에 처리할 수 있는 최대 컨텍스트 길이
            n_ctx=2046,
            # # n_gpu_layers: 실리콘 맥에서는 1이면 충분하다고 한다
            n_gpu_layers=1,
            # n_ctx=512,
            callback_manager=callback_manager,
            # n_batch=512,
            # f16_kv=True,
            # n_threads=16,
            )

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /Users/jominsu/Downloads/llama-2-7b-chat.Q8_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_co

<h2>2. HuggingfaceHub를 사용해서 api형식으로 사용(컴퓨팅파워가 부족할떄 사용!)단, 속도가 느려요.</h2>

In [15]:
# token 발급 받는법: https://huggingface.co/docs/api-inference/quicktour#get-your-api-token

from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass()

In [16]:
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [17]:
from langchain_community.llms import HuggingFaceHub

In [18]:
repo_id = "tiiuae/falcon-40b"

llm = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.5, "max_length": 64}
)

/Users/jominsu/anaconda3/envs/documentChatbot/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


<h1>PDF경로 불러오기</h1>

In [2]:
from langchain.document_loaders import PyPDFLoader
pdf_path = '/Users/jominsu/Desktop/CPPTest/GooglePatent.pdf'
print(pdf_path)

/Users/jominsu/Desktop/CPPTest/GooglePatent.pdf


<h1>textSplit</h1>

In [ ]:
from langchain.text_splitter import  RecursiveCharacterTextSplitter
loader = PyPDFLoader(pdf_path)##경로를 통해 pdf파일 확보
document  = loader.load()##load 파일
text_splitter =  RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=200)## textSplitter정의
chunked_documents = text_splitter.split_documents(document)
chunked_documents

<h1>Embedding</h1>

In [4]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

In [5]:
embeddings = HuggingFaceEmbeddings(
    encode_kwargs={'normalize_embeddings': True}
)

/Users/jominsu/anaconda3/envs/documentChatbot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain.vectorstores import Chroma
vectordb= Chroma.from_documents(chunked_documents, embeddings, persist_directory="./pdfData")
retriever = vectordb.as_retriever()

In [42]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
system_template="""
{summaries}
Always answer in Korean and keep your answers short.
Answer: 
"""

In [43]:
systemprompt = SystemMessagePromptTemplate.from_template(system_template)
humanprompt = HumanMessagePromptTemplate.from_template("{question}")
prompt = ChatPromptTemplate.from_messages([systemprompt,humanprompt])

In [45]:
from langchain.chains import RetrievalQAWithSourcesChain

chain_type_kwargs = {"prompt" : prompt}

chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever =retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)


In [32]:
result = chain("Who is the applicant for this patent?")


Content: US 11,222,252 B2 Page 2 
Related U.S. Application Data 
(60) Provisional application No. 62/009,121 , filed on Jun . 6 , 2014 . 
(56) References Cited 
U.S. PATENT DOCUMENTS 
10,181,098 B2 * 2004/0002848 Al 
2006/0136193 A1 
2014/0229158 A1 1/2019 Vinyals G06F 40/40 
1/2004 Zhou et al . 6/2006 Lux - Pogodalla et al . 8/2014 Zweig 
FOREIGN PATENT DOCUMENTS 
CN 
EP 
EP 
WO 101077011 0094293 0


llama_print_timings:        load time =     482.31 ms
llama_print_timings:      sample time =      46.58 ms /   256 runs   (    0.18 ms per token,  5496.16 tokens per second)
llama_print_timings: prompt eval time =   57088.34 ms /  1421 tokens (   40.17 ms per token,    24.89 tokens per second)
llama_print_timings:        eval time =   25042.31 ms /   255 runs   (   98.21 ms per token,    10.18 tokens per second)
llama_print_timings:       total time =   82866.57 ms /  1676 tokens


In [34]:
result['answer']

'\nContent: US 11,222,252 B2 Page 2 \nRelated U.S. Application Data \n(60) Provisional application No. 62/009,121 , filed on Jun . 6 , 2014 . \n(56) References Cited \nU.S. PATENT DOCUMENTS \n10,181,098 B2 * 2004/0002848 Al \n2006/0136193 A1 \n2014/0229158 A1 1/2019 Vinyals G06F 40/40 \n1/2004 Zhou et al . 6/2006 Lux - Pogodalla et al . 8/2014 Zweig \nFOREIGN PATENT DOCUMENTS \nCN \nEP \nEP \nWO 101077011 0094293 0'

<h1>Test</h1>

In [42]:
res = chain.run({"question":"Who is the applicant for this patent??"})
print(res)

 The applicant for this patent is Google LLC, Mountain View, CA (US).



llama_print_timings:        load time =    7021.36 ms
llama_print_timings:      sample time =       1.93 ms /    20 runs   (    0.10 ms per token, 10384.22 tokens per second)
llama_print_timings: prompt eval time =   22153.49 ms /  1364 tokens (   16.24 ms per token,    61.57 tokens per second)
llama_print_timings:        eval time =  152580.46 ms /    19 runs   ( 8030.55 ms per token,     0.12 tokens per second)
llama_print_timings:       total time =  174768.96 ms /  1383 tokens


<h1>gradio로 챗봇 만들기</h1>

In [ ]:
import gradio as gr
from langchain.schema import AIMessage, HumanMessage, SystemMessage
def response(message, history, additional_input_info):
    
    gpt_response = chain(message)
    print(gpt_response)
    return gpt_response['answer']


gr.ChatInterface(
    fn=response,
    textbox=gr.Textbox(placeholder="질문 해주세요!", container=False, scale=7),
    title="나만의 챗봇을 만든것을 축하드립니다!",
    description="PDF를 기반으로 답변을 해드려요.",
    theme="soft",
    examples=[["특허출원인이 누구야?"], ["출원번호가 뭐야?"], ["청구항1번 내용이뭐야?"]],
    retry_btn="다시보내기 ↩",
    undo_btn="이전챗 삭제 ❌",
    clear_btn="전챗 삭제 💫",
    additional_inputs=[
        gr.Textbox("", label="System Prompt를 입력해주세요", placeholder="I'm lovely chatbot.")
    ]
).launch()